<a href="https://colab.research.google.com/github/Viniciustomko/exercicios/blob/main/allan28_10_Lista_de_Exerc%C3%ADcios_LPA(Allan).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1. Modele, em narrativa e fluxograma, um pipeline de coleta de dados de sensores IoT (coleta → validação → limpeza → armazenamento), incluindo tratamento para leituras ausentes e fora de faixa.

import pandas as pd
import numpy as np
from datetime import datetime, timedelta


timestamps = [datetime.now() - timedelta(minutes=i*5) for i in range(10)]
valores = [20, 25, np.nan, 90, 22, -50, 24, np.nan, 26, 28]
df = pd.DataFrame({'timestamp': timestamps, 'valor': valores})


MIN_TEMP, MAX_TEMP = -40, 85


df['valor'] = df['valor'].where(df['valor'].between(MIN_TEMP, MAX_TEMP))
df['valor'] = df['valor'].interpolate()

print(df)




                   timestamp  valor
0 2025-11-09 18:50:15.610469   20.0
1 2025-11-09 18:45:15.610480   25.0
2 2025-11-09 18:40:15.610483   24.0
3 2025-11-09 18:35:15.610484   23.0
4 2025-11-09 18:30:15.610486   22.0
5 2025-11-09 18:25:15.610488   23.0
6 2025-11-09 18:20:15.610490   24.0
7 2025-11-09 18:15:15.610492   25.0
8 2025-11-09 18:10:15.610493   26.0
9 2025-11-09 18:05:15.610495   28.0


In [ ]:
# 2. Escreva um pseudocódigo para normalizar nomes próprios: remover espaços extras, padronizar para 'Título' (primeira letra maiúscula, demais minúsculas) e eliminar acentos.

import unicodedata

def remover_acentos_unicodedata(texto):
    nfkd_form = unicodedata.normalize('NFD', texto)
    # Remove os caracteres de acentuação (diacríticos)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

# Exemplo de uso
texto_original = "joão gosta de comer feijão, é bom"
texto_sem_acento = remover_acentos_unicodedata(texto_original)
print(texto_sem_acento)



joao gosta de comer feijao, e bom


In [ ]:
#3. Projete um algoritmo para consolidar cadastros duplicados de clientes usando regras: mesma data de nascimento e semelhança de nome ≥ 80% (aproxime a similaridade por contagem de caracteres em comum).

def similaridade(nome1, nome2):
    n1, n2 = set(nome1.lower()), set(nome2.lower())
    return len(n1 & n2) / max(len(n1), len(n2)) if max(len(n1), len(n2)) else 0

def consolidar(clientes):
    consolidados = []
    usados = set()
    for i, c1 in enumerate(clientes):
        if i in usados: continue
        novo = c1.copy()
        for j, c2 in enumerate(clientes):
            if i != j and j not in usados:
                if c1['data_nascimento'] == c2['data_nascimento'] and similaridade(c1['nome'], c2['nome']) >= 0.8:
                    for k, v in c2.items():
                        if not novo.get(k): novo[k] = v
                    usados.add(j)
        consolidados.append(novo)
        usados.add(i)
    return consolidados

# Exemplo
clientes = [
    {'nome': 'Pedro', 'data_nascimento': '1990-06-01', 'email': 'pedro@hotmail.com'},
    {'nome': 'Marina', 'data_nascimento': '1999-01-11', 'email': 'marina12@gmail.com'},
]

print(consolidar(clientes))






[{'nome': 'Pedro', 'data_nascimento': '1990-06-01', 'email': 'pedro@hotmail.com'}, {'nome': 'Marina', 'data_nascimento': '1999-01-11', 'email': 'marina12@gmail.com'}]


In [ ]:
#4. Crie um algoritmo que gere identificadores únicos com verificador: ID base numérico + dígito calculado por soma ponderada (pesos 2..9 cíclicos) e módulo 11.


def id_com_dv(id_base):
    pesos = [2,3,4,5,6,7,8,9]
    s = sum(int(d)*pesos[i%8] for i,d in enumerate(reversed(str(id_base))))
    dv = 11 - s % 11
    return f"{id_base}{0 if dv>=10 else dv}"

# Exemplo
print(id_com_dv(1234568))







12345687


In [ ]:
#5. Elabore um pseudocódigo que valide um formulário web (e-mail, telefone, CEP): para cada campo inválido, acumule mensagem em um vetor de erros e exiba um relatório final.


def validar_formulario(email, telefone, cep):
    erros = []

    if "@" not in email or "." not in email:
        erros.append("E-mail inválido")

    numeros_tel = "".join(filter(str.isdigit, telefone))
    if len(numeros_tel) < 10 or len(numeros_tel) > 11:
        erros.append("Telefone inválido")

    numeros_cep = "".join(filter(str.isdigit, cep))
    if len(numeros_cep) != 8:
        erros.append("CEP inválido")

    if erros:
        print("Erros encontrados:")
        for e in erros:
            print("-", e)
    else:
        print("Formulário válido!")


validar_formulario("viniciuslondon2010@hotmail.com", "(11)90000-0000", "07555-590")




Formulário válido!


In [ ]:
#6. Implemente uma calculadora de demanda com média móvel de 3 períodos (janela deslizante) para prever o próximo valor a partir de um vetor de vendas.


vendas = [120, 200, 225, 240,]

# Média móvel de 3 períodos
medias = [(vendas[i] + vendas[i+1] + vendas[i+2]) / 3 for i in range(len(vendas)-2)]

# Previsão do próximo valor
previsao = medias[-1]

print("Médias móveis:", medias)





Médias móveis: [181.66666666666666, 221.66666666666666]


In [ ]:
#7 Crie um algoritmo que calcule a taxa de conversão de um funil (visitas → leads → vendas) e identifique automaticamente o estágio com maior perda percentual.

def funil(visitas, leads, vendas):
    taxa1 = leads / visitas * 100
    taxa2 = vendas / leads * 100 if leads else 0
    maior_perda = "Visita→Lead" if 100 - taxa1 > 100 - taxa2 else "Lead→Venda"
    return {
        "taxa_visita_para_lead": taxa1,
        "taxa_lead_para_venda": taxa2,
        "taxa_visita_para_venda": vendas / visitas * 100,
        "maior_perda": maior_perda
    }

# Exemplo
print(funil(1000, 200, 50))

{'taxa_visita_para_lead': 20.0, 'taxa_lead_para_venda': 25.0, 'taxa_visita_para_venda': 5.0, 'maior_perda': 'Visita→Lead'}


In [ ]:
#8. Escreva um algoritmo que, dado um intervalo de datas (dia inicial e final) e uma lista de feriados fixos (dd/mm), conte quantos 'dias letivos' existem excluindo sábados, domingos e feriados.

from datetime import date, timedelta

ini = date(2025, 3, 1)
fim = date(2025, 3, 31)
feriados = {"12/10"}

dias = sum(
    1 for i in range((fim - ini).days + 1)
    if (d := ini + timedelta(i)).weekday() < 5 and d.strftime("%d/%m") not in feriados
)

print(f"Dias letivos: {dias}")
print(f"feriados: {feriados}")


Dias letivos: 21
feriados: {'12/10'}


In [ ]:
#9. Desenvolva um gerador de boletim de medidores de energia: para cada medidor, calcule consumo (leitura_atual − leitura_anterior), sinalize anomalia se consumo > média + 2*desvio padrão (aproxime o desvio usando apenas média das diferenças).

import statistics

def gerar_boletim_curto(dados: list[dict]):

    consumos = [d['leitura_atual'] - d['leitura_anterior'] for d in dados]

    if not consumos:
        return dados


    media = statistics.mean(consumos)
    desvio_padrao = statistics.stdev(consumos)
    limiar_anomalia = media + (2 * desvio_padrao)


    for d, consumo in zip(dados, consumos):
        d['consumo'] = consumo
        d['anomalia'] = consumo > limiar_anomalia
        d['status'] = "ANOMALIA" if d['anomalia'] else "Normal"
        d['limiar'] = f"{limiar_anomalia:.2f}"

    return dados


dados_exemplo = [
    {'id': 'M001', 'leitura_anterior': 260, 'leitura_atual': 280},
    {'id': 'M002', 'leitura_anterior': 200, 'leitura_atual': 300}

]

boletim = gerar_boletim_curto(dados_exemplo)

for item in boletim:
    print(item)

{'id': 'M001', 'leitura_anterior': 260, 'leitura_atual': 280, 'consumo': 20, 'anomalia': False, 'status': 'Normal', 'limiar': '173.14'}
{'id': 'M002', 'leitura_anterior': 200, 'leitura_atual': 300, 'consumo': 100, 'anomalia': False, 'status': 'Normal', 'limiar': '173.14'}


In [ ]:
#10. Crie um algoritmo para cálculo de frete multimodal: some custos por trecho (rodoviário/ferroviário/aéreo), aplique adicional por peso acima de 30kg e desconto por distância acima de 800km.

def calcular_frete(trechos, peso):
    custo = sum(t['custo'] for t in trechos)
    distancia = sum(t['distancia'] for t in trechos)

    if peso > 30:
        custo *= 1.10  # +10% se peso > 30kg
    if distancia > 800:
        custo *= 0.95  # -5% se distância > 800km

    return round(custo, 2)

# Exemplo
trechos = [
    {'tipo': 'rodoviario', 'custo': 200, 'distancia': 300},
    {'tipo': 'ferroviario', 'custo': 100, 'distancia': 400},
    {'tipo': 'aereo', 'custo': 60, 'distancia': 200}
]
peso = 35

print(f"Frete: R$ {calcular_frete(trechos, peso)}")













Frete: R$ 376.2


In [ ]:
#11. Simule a lógica de despacho de elevador para um prédio de 10 andares: escolha a próxima chamada minimizando a distância (varra na direção atual; ao fim, inverta a direção).

andar_atual, direcao = 1, 1
chamadas = {3, 7, 2, 9}

while chamadas:

    chamadas_dir = [f for f in chamadas if (f - andar_atual) * direcao >= 0]
    if not chamadas_dir:
        direcao *= -1
        chamadas_dir = [f for f in chamadas if (f - andar_atual) * direcao >= 0]


    proximo_andar = min(chamadas_dir, key=lambda f: abs(f - andar_atual))
    andar_atual = proximo_andar
    chamadas.remove(proximo_andar)
    print(f"Andar {andar_atual}, direção {'subindo' if direcao==1 else 'descendo'}")






Andar 2, direção subindo
Andar 3, direção subindo
Andar 7, direção subindo
Andar 9, direção subindo


In [ ]:
#12. Implemente uma seleção de propostas de fornecedores: dado um vetor de preços e um vetor de prazos, classifique cada proposta por escore = 0.7*(preço_min/preço) + 0.3*(prazo_min/prazo) e escolha a melhor.

def selecionar_melhor_fornecedor(precos, prazos):
    # Garantir que os vetores tenham o mesmo tamanho
    if len(precos) != len(prazos):
        raise ValueError("Os vetores de preços e prazos devem ter o mesmo tamanho.")

    preco_min = min(precos)
    prazo_min = min(prazos)

    escores = [
        0.7 * (preco_min / preco) + 0.3 * (prazo_min / prazo)
        for preco, prazo in zip(precos, prazos)
    ]

    melhor_indice = escores.index(max(escores))
    return melhor_indice, escores[melhor_indice]

precos = [50,80,100]
prazos = [15,10,12]

indice, escore = selecionar_melhor_fornecedor(precos, prazos)
print(f"Melhor fornecedor: {indice} (escore = {escore:.4f})")



Melhor fornecedor: 0 (escore = 0.9000)


In [ ]:
#13. Escreva um algoritmo que classifique automaticamente a prioridade de tickets de suporte (baixa/média/alta) combinando palavras-chave no texto (ex.: 'erro', 'parado') e impacto informado (1 a 5).

def classificar_prioridade(texto_ticket, impacto):



    texto = texto_ticket.lower()


    palavras_alta = ["erro", "falha", "parado", "urgente", "crítico", "inacessível"]
    palavras_media = ["lento", "intermitente", "demora", "problema", "bug"]


    pontuacao = 0
    for palavra in palavras_alta:
        if palavra in texto:
            pontuacao += 2
    for palavra in palavras_media:
        if palavra in texto:
            pontuacao += 1


    pontuacao_total = pontuacao + impacto


    if pontuacao_total >= 6:
        prioridade = "Alta"

    else:
        prioridade = "media"

    return prioridade



tickets = [
    ("O sistema deu erro", 7),
    ("bug na Aplicação ", 5),
    ("problema na interface", 4),
    ("demora no sistema", 3),
]

for texto, impacto in tickets:
    print(f"Ticket: {texto}")
    print(f"Impacto: {impacto} → Prioridade: {classificar_prioridade(texto, impacto)}\n")

Ticket: O sistema deu erro
Impacto: 7 → Prioridade: Alta

Ticket: bug na Aplicação 
Impacto: 5 → Prioridade: Alta

Ticket: problema na interface
Impacto: 4 → Prioridade: media

Ticket: demora no sistema
Impacto: 3 → Prioridade: media



In [ ]:
#14. Crie um verificador de integridade de dados por amostragem: selecione 10% dos registros de forma pseudoaleatória (use gerador linear congruente) e reporte quantos possuem campos nulos.

import pandas as pd

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    while True:
        seed = (a * seed + c) % m
        yield seed / m

def verificar_integridade(df, proporcao=0.1, seed=123):
    n = len(df)
    amostra = int(n * proporcao)
    rng = lcg(seed)
    idx = [int(next(rng) * n) for _ in range(amostra)]
    amostra_df = df.iloc[idx]
    nulos = amostra_df.isnull().any(axis=1).sum()
    print(f"Total: {n}, Amostra: {amostra}, Registros com nulos: {nulos}")


df = pd.DataFrame({
    "id": range(1, 21),
    "nome": ["Ana", None, "João", "Bia", "Leo"] * 4,
    "idade": [23, 30, None, 40, 19] * 4
})

verificar_integridade(df)

Total: 20, Amostra: 2, Registros com nulos: 0


In [ ]:
#15. Elabore um pseudocódigo que atribua turmas a salas: dadas turmas com tamanhos e salas com capacidades, aloque sem ultrapassar capacidade e minimize salas ociosas (ganancioso: ordenar turmas desc e encaixar).

turmas = [20,30,50]
salas = [20,40,60]

turmas.sort(reverse=True)       # maior para menor
salas.sort()                    # menor para maior

alocacoes = []
usadas = [False]*len(salas)

for t in turmas:
    for i, s in enumerate(salas):
        if not usadas[i] and s >= t:
            alocacoes.append((t, s))
            usadas[i] = True
            break
    else:
        alocacoes.append((t, None))

print(alocacoes)

[(50, 60), (30, 40), (20, 20)]


In [ ]:
#16. Simule um call center com duas filas (prioritária e comum): para cada minuto, atenda até N chamadas priorizando a fila prioritária; ao final, reporte tempo médio de espera por fila.

import random, collections

fila_p = collections.deque()
espera_p = [5]

for t in range(120):
    if random.random()<0.3: fila_p.append(t)

    for _ in range(3):
        if fila_p: espera_p.append(t - fila_p.popleft())

        else: break

print(f"Prioritária: {sum(espera_p)/len(espera_p):.2f} min, Comum: {sum(espera_p)/len(espera_p):.2f} min")



Prioritária: 0.11 min, Comum: 0.11 min


In [ ]:
#17 Implemente compressão simples RLE (Run-Length Encoding) para textos: comprima sequências de caracteres repetidos (ex.: 'AAABB' → '3A2B') e descomprima o resultado.


def comprimir(texto):
    resultado = ""
    cont = 1
    for i in range(1, len(texto)):
        if texto[i] == texto[i - 1]:
            cont += 1
        else:
            resultado += str(cont) + texto[i - 1]
            cont = 1
    if texto:
        resultado += str(cont) + texto[-1]
    return resultado

# Descompressão RLE
def descomprimir(codificado):
    resultado = ""
    num = ""
    for c in codificado:
        if c.isdigit():
            num += c
        else:
            resultado += c * int(num)
            num = ""
    return resultado


texto = "vinicius"
comp = comprimir(texto)
decomp = descomprimir(comp)

print("Original:", texto)
print("Comprimido:", comp)
print("Descomprimido:", decomp)


Original: vinicius
Comprimido: 1v1i1n1i1c1i1u1s
Descomprimido: vinicius


In [ ]:
#18. Crie um algoritmo para cifra de César com chave variável k (1..25): criptografe e descriptografe preservando espaços e pontuação.

def cifra_cesar(texto, k):
    res = ""
    for c in texto:
        if c.isalpha():
            base = ord('A') if c.isupper() else ord('a')
            res += chr((ord(c) - base + k) % 26 + base)
        else:
            res += c
    return res


msg = "hello guys"
k = 2
cripto = cifra_cesar(msg, k)
decripto = cifra_cesar(cripto, -k)

print("Criptografado:", cripto)
print("Descriptografado:", decripto)


Criptografado: jgnnq iwau
Descriptografado: hello guys


In [ ]:
#19 Desenvolva um analisador de logs: receba N linhas 'DATA;SERVIÇO;SEVERIDADE;MENSAGEM', conte ocorrências por severidade e identifique as 3 mensagens mais frequentes


from collections import Counter

linhas = [
    "2025-11-06;auth;ERROR;Falha ao autenticar usuário",
    "2025-11-06;db;INFO;Conexão estabelecida",
    "2025-11-06;auth;WARNING;Tentativa de login inválida",
    "2025-11-06;auth;ERROR;Falha ao autenticar usuário",
    "2025-11-06;api;INFO;Requisição processada",
    "2025-11-06;auth;ERROR;Falha ao autenticar usuário",
    "2025-11-06;db;ERROR;Timeout de conexão",
    "2025-11-06;api;INFO;Requisição processada",
]

severidades = Counter()
mensagens = Counter()

for l in linhas:
    _, _, sev, msg = l.split(";")
    severidades[sev] += 1
    mensagens[msg] += 1

print("Por severidade:", dict(severidades))
print("Top 3 mensagens:", mensagens.most_common(3))


Por severidade: {'ERROR': 4, 'INFO': 3, 'WARNING': 1}
Top 3 mensagens: [('Falha ao autenticar usuário', 3), ('Requisição processada', 2), ('Conexão estabelecida', 1)]


In [ ]:
#20. Escreva um algoritmo que verifique se uma expressão tem parênteses, colchetes e chaves balanceados (use um vetor simulando pilha).
def balanceada(exp):
    pilha = []
    pares = {')': '(', ']': '[', '}': '{'}
    for s in exp:
        if s in '([{': pilha.append(s)
        elif s in ')]}':
            if not pilha or pilha.pop() != pares[s]:
                return False
    return not pilha

# Exemplo de uso
print(balanceada("([]{})"))   # True colchetes corretos
print(balanceada("([)]"))     # False
print(balanceada("([]"))   #false
print(balanceada("()")) #true
print(balanceada(")(")) #false


True
False
False
True
False


In [ ]:
#21 Mescle dois vetores já ordenados em um único vetor ordenado sem usar ordenação pronta (varra com dois índices).

def merge_sorted_vectors(v1, v2):
    merged, i, j = [], 0, 0
    while i < len(v1) and j < len(v2):
        if v1[i] < v2[j]:
            merged.append(v1[i]); i += 1
        else:
            merged.append(v2[j]); j += 1
    merged.extend(v1[i:] or v2[j:])
    return merged

# Exemplo
vetor1 = [1,3,9]
vetor2 = [2,4,]
print(merge_sorted_vectors(vetor1, vetor2))

[1, 2, 3, 4, 9]


In [ ]:
#22  Encontre a mediana de um vetor sem ordenar totalmente: utilize seleção por partição (particione e decida o lado até achar a posição alvo).

import random

def particao(vetor, inicio, fim):
    pivo = vetor[fim]
    i = inicio
    for j in range(inicio, fim):
        if vetor[j] < pivo:
            vetor[i], vetor[j] = vetor[j], vetor[i]
            i += 1
    vetor[i], vetor[fim] = vetor[fim], vetor[i]
    return i

def selecao_rapida(vetor, k):
    inicio, fim = 0, len(vetor) - 1
    while True:
        indice_pivo = particao(vetor, inicio, fim)
        if k == indice_pivo:
            return vetor[k]
        elif k < indice_pivo:
            fim = indice_pivo - 1
        else:
            inicio = indice_pivo + 1

def media(vetor):
    n = len(vetor)
    if n % 2 == 1:
        return selecao_rapida(vetor, n // 2)
    else:
        return (selecao_rapida(vetor, n // 2 - 1) + selecao_rapida(vetor, n // 2)) / 2

# Exemplo
vetor = [10,10]
print("Media:", media(vetor))

Media: 10.0


In [ ]:
#23. Implemente busca binária em vetor ordenado retornando também o número de comparações realizadas.

def busca_binaria(vetor, alvo):


    inicio = 0
    fim = len(vetor) - 1
    comparacoes = 0

    while inicio <= fim:
        meio = (inicio + fim) // 2
        comparacoes += 1

        if vetor[meio] == alvo:
            return meio, comparacoes
        elif vetor[meio] < alvo:
            inicio = meio + 1
        else:
            fim = meio - 1

    return -1, comparacoes



valores = [1, 2, 4, 5]
indice, comps = busca_binaria(valores, 4)
print(f"Índice encontrado: {indice}, Comparações: {comps}")



Índice encontrado: 2, Comparações: 2


In [ ]:
# 24. Rotacione uma matriz quadrada N×N 90 graus no sentido horário, in-place (transposição + reversão de linhas).
def rotacionar_90(m):
    n = len(m)
    for i in range(n):
        for j in range(i + 1, n):
            m[i][j], m[j][i] = m[j][i], m[i][j]  # transpor
    for linha in m:
        linha.reverse()


m = [[11,22,33],[40,50,60]]
rotacionar_90(m)
print(m)




[[33, 40, 11], [60, 50, 22]]


In [ ]:
#25 Calcule o caminho de menor custo em uma grade N×M com movimentos apenas para direita ou baixo, usando programação dinâmica (tabela de custos cumulativos).

def menor_caminho(g):
    n, m = len(g), len(g[0])
    for i in range(1, n): g[i][0] += g[i-1][0]
    for j in range(1, m): g[0][j] += g[0][j-1]
    for i in range(1, n):
        for j in range(1, m):
            g[i][j] += min(g[i-1][j], g[i][j-1])
    return g[-1][-1]


grid = [[1,3,1],[1,5,1],[4,2,1]]
print(menor_caminho(grid))

7


In [ ]:
#26 Implemente um gerador de números pseudoaleatórios por congruência linear: X_{n+1} = (a*X_n + c) mod m; escreva funções para inicializar semente e gerar próximo valor.

a, c, m = 1664525, 1013904223, 2**32
X = 1  # semente padrão

def inicializar_semente(seed):
    global X
    X = seed % m

def gerar_proximo():
    global X
    X = (a * X + c) % m
    return X

inicializar_semente(42)
for _ in range(5):
    print(gerar_proximo())

1083814273
378494188
2479403867
955863294
1613448261


In [ ]:
#27 Crie funções para converter números entre bases 2, 8, 10 e 16 (sem usar conversores prontos), incluindo validação de dígitos.

def validar(n, b):
    return all(c in "0123456789ABCDEF"[:b] for c in n.upper())

def para_dec(n, b):
    if not validar(n, b): raise ValueError("Dígito inválido")
    v, d = 0, "0123456789ABCDEF"
    for c in n.upper(): v = v * b + d.index(c)
    return v

def de_dec(n, b):
    if n == 0: return "0"
    d, r = "0123456789ABCDEF", ""
    while n > 0:
        r = d[n % b] + r
        n //= b
    return r

def converter(n, b1, b2):
    return de_dec(para_dec(n, b1), b2)


print(converter("1011", 2, 10))
print(converter("17", 8, 2))
print(converter("2F", 16, 10))
print(converter("255", 10, 16))

11
1111
47
FF


In [ ]:
#28 Escreva uma função que compute o dígito verificador de boleto (módulo 11 com pesos 2..9) e outra que valide uma linha digitável completa.

def modulo11_dv(n):
    p, s = 2, 0
    for d in reversed(n):
        s += int(d) * p
        p = 2 if p == 9 else p + 1
    r = 11 - s % 11
    return 1 if r in (0, 10, 11) else r

def validar_boleto(linha):
    linha = ''.join(filter(str.isdigit, linha))
    if len(linha) != 47:
        return False
    dv = int(linha[4])
    return dv == modulo11_dv(linha[:4] + linha[5:])

boleto = "000000001111122123"
print(validar_boleto(boleto))  # True ou False




False


In [ ]:
# 29 Dado um conjunto de tarefas com início e fim (hh:mm), selecione o maior número de tarefas não sobrepostas (algoritmo ganancioso por menor horário de término).

tarefas = [("10:00", "10:30"), ("10:00", "11:00"), ("12:00", "13:30")]


h2m = lambda h: int(h[:2])*60 + int(h[3:])


tarefas.sort(key=lambda x: h2m(x[1]))
fim_atual = 0
selecionadas = []
for i, f in tarefas:
    if h2m(i) >= fim_atual:
        selecionadas.append((i, f))
        fim_atual = h2m(f)

print("Tarefas selecionadas:", selecionadas)

Tarefas selecionadas: [('10:00', '10:30'), ('12:00', '13:30')]


In [ ]:
# 30. Gere uma tabela alinhada de dados a partir de texto CSV (separado por ';'): calcule a largura máxima de cada coluna e imprima as linhas com espaçamento adequado.

csv_text = """Nome;Idade;Cidade
Aline;20;Madrid
Pedro;30;Lisboa
Jean;40;Paris"""

linhas = [linha.split(';') for linha in csv_text.splitlines()]
larguras = [max(len(linha[i]) for linha in linhas) for i in range(len(linhas[0]))]

for linha in linhas:
    print(" | ".join(linha[i].ljust(larguras[i]) for i in range(len(linha))))

Nome  | Idade | Cidade
Aline | 20    | Madrid
Pedro | 30    | Lisboa
Jean  | 40    | Paris 
